In [75]:
import os
import json
import pandas as pd
import numpy as np

# get all json files that store our interactions with chatgpt
# these are the final, optimized files
json_files_final_run = [
    os.path.join(dp, f) for dp, dn, fn in os.walk("../data/prompts_new_format") for f in fn
    if ("final" in f) and f.endswith(".json") and (not f.endswith(" 2.json"))]

total_files = len(json_files_final_run)

# read all the data into a pandas dataframe
li = []
for filename in json_files_final_run:
    json_data = json.load(open(filename))
    for i, iteration in enumerate(json_data):
        iteration["iteration"] = i
        iteration["file"] = filename
        df = pd.DataFrame(columns=["iteration", "prompt", "code", "tfsec", "number_of_issues", "file"])
        df = df.append(iteration, ignore_index=True)
        li.append(df)
frame_all_runs = pd.concat(li, axis=0, ignore_index=True)

/var/folders/bq/l2q_cb9j6bj21nlxmzykz44r0000gn/T/ipykernel_6793/1741334272.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(iteration, ignore_index=True)
/var/folders/bq/l2q_cb9j6bj21nlxmzykz44r0000gn/T/ipykernel_6793/1741334272.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(iteration, ignore_index=True)
/var/folders/bq/l2q_cb9j6bj21nlxmzykz44r0000gn/T/ipykernel_6793/1741334272.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(iteration, ignore_index=True)
/var/folders/bq/l2q_cb9j6bj21nlxmzykz44r0000gn/T/ipykernel_6793/1741334272.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

In [76]:
print(len(frame_all_runs))
frame_all_runs["number_of_issues"].replace("", np.nan)
frame_all_runs["number_of_issues"].dropna(inplace=True)
frame_all_runs = frame_all_runs[frame_all_runs["number_of_issues"].astype(bool)]
frame_all_runs["number_of_issues"] = frame_all_runs["number_of_issues"].astype(int)
frame_all_runs = frame_all_runs[frame_all_runs["number_of_issues"] >= 0]
# len(sequence) > 1) or (sequence[0] == 0)
print(len(frame_all_runs))
print(frame_all_runs.dtypes)

213
164
iteration           object
prompt              object
code                object
tfsec               object
number_of_issues     int64
file                object
dtype: object


In [77]:
frame_all_runs.groupby("iteration").mean()

/var/folders/bq/l2q_cb9j6bj21nlxmzykz44r0000gn/T/ipykernel_6793/1746645134.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  frame_all_runs.groupby("iteration").mean()


,number_of_issues
iteration,
0,3.987500
1,2.693878
2,3.578947
3,4.846154
4,3.500000
5,0.000000


In [55]:
frame_all_runs.groupby("iteration").std()

/var/folders/bq/l2q_cb9j6bj21nlxmzykz44r0000gn/T/ipykernel_6793/3127731910.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  frame_all_runs.groupby("iteration").std()


,number_of_issues
iteration,
0,4.026789
1,5.264678
2,4.500812
3,6.216191
4,4.949747
5,NaN


In [54]:
# extract all issue sequences per file
files = frame_all_runs["file"].unique()
files

array(['../data/prompts_new_format/prompt4/versuch14/final_1.json',
       '../data/prompts_new_format/prompt4/versuch13/final_0.json',
       '../data/prompts_new_format/prompt4/abbruch1/prompt_4_final_1.json',
       '../data/prompts_new_format/prompt4/versuch12/final_1.json',
       '../data/prompts_new_format/prompt4/versuch15/final_1.json',
       '../data/prompts_new_format/prompt4/versuch8/prompt_4_final_1.json',
       '../data/prompts_new_format/prompt4/versuch6/prompt_4_final_2.json',
       '../data/prompts_new_format/prompt4/versuch7/prompt_4_final_1.json',
       '../data/prompts_new_format/prompt4/versuch9/prompt_4_final_4.json',
       '../data/prompts_new_format/prompt4/versuch10/prompt_4_final_1.json',
       '../data/prompts_new_format/prompt4/versuch11/final_1.json',
       '../data/prompts_new_format/prompt4/versuch2/prompt_4_final_2.json',
       '../data/prompts_new_format/prompt4/versuch5/prompt_4_final_1.json',
       '../data/prompts_new_format/prompt4/versuch4

In [58]:
sequences = []
for file in files:
    slice = frame_all_runs[frame_all_runs["file"].str.contains(file)]
    slice.sort_values("iteration")
    sequences.append(slice["number_of_issues"].to_list())

In [71]:
pd.DataFrame([" ".join(str(sequence)) for sequence in sequences], columns=["sequence"]).value_counts()

sequence                           
[ 2 ,   0 ]                            15
[ 2 ]                                  10
[ 0 ]                                   9
[ 2 ,   1 ]                             7
[ 4 ,   0 ]                             2
[ 1 1 ]                                 2
[ 2 ,   3 ,   1 ,   1 ]                 2
[ 7 ]                                   2
[ 4 ]                                   2
[ 8 ]                                   2
[ 9 ,   7 ,   5 ,   5 ]                 2
[ 2 ,   1 ,   0 ]                       2
[ 9 ,   6 ,   6 ,   6 ,   7 ,   0 ]     1
[ 9 ,   1 ,   1 ,   0 ]                 1
[ 9 ,   7 ,   6 ,   5 ]                 1
[ 7 ,   4 ]                             1
[ 7 ,   2 ,   3 ,   2 ,   0 ]           1
[ 9 ,   7 ]                             1
[ 6 ]                                   1
[ 6 ,   2 ]                             1
[ 5 ]                                   1
[ 1 3 ,   1 6 ,   1 5 ,   1 7 ]         1
[ 4 ,   1 ]                             

# Success and error rates

In [107]:
absolute_success = [sequence for sequence in sequences if sequence[-1] == 0]
print(f"Reduction to 0 vulnerabilities: {(len(absolute_success)/len(sequences))*100}%")


Reduction to 0 vulnerabilities: 41.25%


In [100]:
improvement = [sequence for sequence in sequences if sequence[-1] < sequence[0]]
print(f"Reduction in number of vulnerabilities: {(len(improvement)/len(sequences))*100}%")


Reduction in number of vulnerabilities: 53.75%


In [101]:
fails = [sequence for sequence in sequences if len(sequence) == 1]
print(f"Some error (ChatGPT did not return code) {(len(fails) / len(sequences))*100}%")

Some error (ChatGPT did not return code) 38.75%


In [102]:
worse = [sequence for sequence in sequences if sequence[-1] > sequence[0]]
print(f"ChatGPT worsened security {(len(worse) / len(sequences))*100}%")

ChatGPT worsened security 5.0%


In [104]:
nothing = [sequence for sequence in sequences if sequence[-1] == sequence[0] and len(sequence) > 1]
print(f"ChatGPT had no effect {(len(nothing) / len(sequences))*100}%")

ChatGPT had no effect 2.5%


In [111]:
iterations_to_zero = sum([len(sequence) for sequence in absolute_success])/len(absolute_success)
print(f"On average it takes {iterations_to_zero} iterations to reach 0 vulnerabilities")

On average it takes 2.1515151515151514 iterations to reach 0 vulnerabilities
